In [30]:
import requests
import pprint
import sys
import math
import pandas as pd
pp = pprint.PrettyPrinter(indent=4)

In [31]:
### OPTIONS #####
# --------------------------------------------
SCENARIO = "peak_heat_day"  # ["typical_heat_day", "peak_heat_day"]

# set this flag to True for running with service, False for running with local
SERVICE = True  # False to run locally
TESTCASE = "Adrenalin1"  # ["Adrenalin1", "'time': (0, 31536000),"]
NAMESPACE = "Adrenalin_training"

In [32]:
# Set URL for testcase and get test_id
if SERVICE: 
    url = "https://api.boptest.net"
    # select testcase and get id
    res = requests.post(f"{url}/testcases/{NAMESPACE}/{TESTCASE}/select")
    test_id = f"/{res.json()['testid']}"  # add / for url compatablity
else:
    url = "http://127.0.0.1:5000"
    test_id = ""

In [33]:
test_id

'/68e11392-2f48-4e4a-898e-08487f5ac0ee'

In [34]:
name = requests.get(f"{url}/name{test_id}").json()["payload"]["name"]
print(name)

Adrenalin1_training


In [35]:
inputs = requests.get(f"{url}/inputs{test_id}").json()["payload"]
print("Control Inputs:")
pp.pprint(inputs)

Control Inputs:
{   'ahu_oveFanRet_activate': {   'Description': 'Activation for AHU return '
                                                 'fan speed control signal',
                                  'Maximum': None,
                                  'Minimum': None,
                                  'Unit': None},
    'ahu_oveFanRet_u': {   'Description': 'AHU return fan speed control signal',
                           'Maximum': 1,
                           'Minimum': 0,
                           'Unit': '1'},
    'ahu_oveFanSup_activate': {   'Description': 'Activation for AHU supply '
                                                 'fan speed control signal',
                                  'Maximum': None,
                                  'Minimum': None,
                                  'Unit': None},
    'ahu_oveFanSup_u': {   'Description': 'AHU supply fan speed control signal',
                           'Maximum': 1,
                           'Minimum': 0,
     

In [36]:
measurements = requests.get(f"{url}/measurements{test_id}").json()["payload"]
print("Measurements:")
pp.pprint(measurements)

Measurements:
{   'ahu_reaFloExtAir_y': {   'Description': 'AHU extract air volume flowrate',
                              'Maximum': None,
                              'Minimum': None,
                              'Unit': 'm3/s'},
    'ahu_reaFloSupAir_y': {   'Description': 'AHU supply air volume flowrate',
                              'Maximum': None,
                              'Minimum': None,
                              'Unit': 'm3/s'},
    'ahu_reaTCoiSup_y': {   'Description': 'AHU heating coil supply water '
                                           'temperature',
                            'Maximum': None,
                            'Minimum': None,
                            'Unit': 'K'},
    'ahu_reaTHeaRec_y': {   'Description': 'AHU air temperature exiting heat '
                                           'recovery in supply air stream',
                            'Maximum': None,
                            'Minimum': None,
                            'Unit'

In [37]:
forecasts = requests.get(f"{url}/forecast_points{test_id}").json()["payload"]
print("Forecasts:")
pp.pprint(forecasts)

Forecasts:
{   'EmissionsBiomassPower': {   'Description': 'Kilograms of carbon dioxide '
                                                'to produce 1 kWh thermal from '
                                                'biomass',
                                 'Unit': 'kgCO2/kWh'},
    'EmissionsDistrictHeatingPower': {   'Description': 'Kilograms of carbon '
                                                        'dioxide to produce 1 '
                                                        'kWh thermal district '
                                                        'heating',
                                         'Unit': 'kgCO2/kWh'},
    'EmissionsElectricPower': {   'Description': 'Kilograms of carbon dioxide '
                                                 'to produce 1 kWh of '
                                                 'electricity',
                                  'Unit': 'kgCO2/kWh'},
    'EmissionsGasPower': {   'Description': 'Kilograms of carbon dioxide t

In [1]:
import torch

def enable_gpu():
    # Check if GPU is available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU is available. Using:", torch.cuda.get_device_name(0))
    else:
        device = torch.device("cpu")
        print("GPU is not available. Using CPU.")
    
    return device

def main():
    # Enable GPU
    device = enable_gpu()

    # Create a simple tensor
    x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
    
    # Move tensor to GPU if available
    x = x.to(device)
    
    # Perform a simple operation
    y = x * 2
    
    print("Input tensor:", x)
    print("Output tensor:", y)
    print("Device tensor is on:", x.device)

if __name__ == "__main__":
    main()

GPU is available. Using: NVIDIA T600 Laptop GPU
Input tensor: tensor([1., 2., 3., 4., 5.], device='cuda:0')
Output tensor: tensor([ 2.,  4.,  6.,  8., 10.], device='cuda:0')
Device tensor is on: cuda:0


In [8]:
# -*- coding: utf-8 -*-
"""
Template script for the Adrenalin competition


"""


import requests
import pprint
import sys
import math
import pandas as pd

pp = pprint.PrettyPrinter(indent=4)

### OPTIONS #####
# --------------------------------------------
SCENARIO = "peak_heat_day"  # ["typical_heat_day", "peak_heat_day"]

# set this flag to True for running with service, False for running with local
SERVICE = False  # False to run locally
TESTCASE = "singlezone_commercial_hydronict"  # ["Adrenalin1", "Adrenalin2"]
NAMESPACE = "Adrenalin_training"

# controller settings
dt = 3600  # control step
horizon = 24 * dt  # forecast horizon
forecast_points = ["TDryBul"]  # set forecast points needed for controller

# Set URL for testcase and get test_id
if SERVICE: 
    url = "https://api.boptest.net"
    # select testcase and get id
    res = requests.post(f"{url}/testcases/{NAMESPACE}/{TESTCASE}/select")
    test_id = f"/{res.json()['testid']}"  # add / for url compatablity
else:
    url = "http://127.0.0.1:5000"
    test_id = ""


# GET TEST CASE INFORMATION
# --------------------------------------------
print("\nTEST CASE INFORMATION\n---------------------")
# Get the test case name
name = requests.get(f"{url}/name{test_id}").json()["payload"]["name"]
print("\nName is:")
pp.pprint(name)
if SERVICE:
    print(f"test_id={test_id.removeprefix('/')}")

# Retrieve a list of inputs (controllable points) for the model from REST API
inputs = requests.get(f"{url}/inputs{test_id}").json()["payload"]
print("Control Inputs:")
pp.pprint(inputs)
# Retrieve a list of measurements (outputs) for the model from REST API
measurements = requests.get(f"{url}/measurements{test_id}").json()["payload"]
print("Measurements:")
pp.pprint(measurements)
# Retrieve a list of available forecast points for the model from REST API
forecasts = requests.get(f"{url}/forecast_points{test_id}").json()["payload"]
print("Forecasts:")
pp.pprint(forecasts)

# INITIALIZE TEST CASE
# --------------------------------------------
# Set controller timestep
res = requests.put(f"{url}/step{test_id}", json={"step": dt})

# set scenario. Takes some time as the testcase is initializing
scenData = {"electricity_price": "highly_dynamic", "time_period": SCENARIO}
res = requests.put(f"{url}/scenario{test_id}", json=scenData).json()

# get measurements at inital state
y = res["payload"]["time_period"]
# get current time
start_time = y["time"]


# RUN SCENARIO
# --------------------------------------------
Nsim = math.ceil(24 * 14 * 3600 / dt)  # run for two weeks
for i in range(Nsim):
    # get forecast
    forecast = requests.put(
        f"{url}/forecast{test_id}",
        json={"point_names": forecast_points, "horizon": horizon, "interval": dt},
    ).json()["payload"]

    ### implement your controller magic #####

    # set you controller inputs in the u dictionary. Remember input activation
    u = {"oveTSupSet_u":300.}  # Empty dict means no changes in input

    # advance one control step
    res = requests.post(f"{url}/advance{test_id}", json=u).json()

    if res["status"] == 200:
        print(f"\rSimulated step {i+1} of {Nsim}", file=sys.stdout, end="", flush=True)
        # get measurements after control step
        y = res["payload"]
    else:
        break

final_time = start_time + Nsim * dt


# GET RESULTS
# -----------------------------------
# get measurement results
res_mea = requests.put(
    f"{url}/results{test_id}",
    json={
        "point_names": list(measurements.keys()),
        "start_time": start_time,
        "final_time": final_time,
    },
)
df_mea = pd.DataFrame.from_dict(res_mea.json()['payload']).set_index('time')

# get input results
res_inp = requests.put(
    f"{url}/results{test_id}",
    json={
        "point_names": list(inputs.keys()),
        "start_time": start_time,
        "final_time": final_time,
    },
)
df_inp = pd.DataFrame.from_dict(res_inp.json()['payload'])

# merge result dataframes
df_res = pd.concat([df_mea, df_inp], axis=1)

# Report KPIs
kpi = requests.get(f"{url}/kpi{test_id}").json()["payload"]
print("\nKPIs are:")
pp.pprint(kpi)


if SERVICE:
    stop_response = requests.put(f"{url}/stop{test_id}")
    if stop_response.status_code == 200:
        print("Successfully stopped the test!")
    else:
        print(f"Couldn't stop test with status code: {stop_response.status_code}!")


TEST CASE INFORMATION
---------------------

Name is:
'singlezone_commercial_hydronic'
Control Inputs:
{   'ahu_oveFanRet_activate': {   'Description': 'Activation for AHU return '
                                                 'fan speed control signal',
                                  'Maximum': None,
                                  'Minimum': None,
                                  'Unit': None},
    'ahu_oveFanRet_u': {   'Description': 'AHU return fan speed control signal',
                           'Maximum': 1.0,
                           'Minimum': 0.0,
                           'Unit': '1'},
    'ahu_oveFanSup_activate': {   'Description': 'Activation for AHU supply '
                                                 'fan speed control signal',
                                  'Maximum': None,
                                  'Minimum': None,
                                  'Unit': None},
    'ahu_oveFanSup_u': {   'Description': 'AHU supply fan speed control sign

In [2]:
import sys
from collections import OrderedDict
sys.path.insert(0,'boptestGymService')
from boptestGymEnv import BoptestGymEnv, NormalizedObservationWrapper, DiscretizedActionWrapper
from stable_baselines3 import DQN

# url for the BOPTEST service. 
url = 'https://api.boptest.net' 

# Decide the state-action space of your test case
env = BoptestGymEnv(
        url                  = url,
        testcase             = 'bestest_hydronic_heat_pump',
        actions              = ['oveHeaPumY_u'],
        observations         = {'time':(0,604800),
                                'reaTZon_y':(280.,310.),
                                'TDryBul':(265,303),
                                'HDirNor':(0,862),
                                'InternalGainsRad[1]':(0,219),
                                'PriceElectricPowerHighlyDynamic':(-0.4,0.4),
                                'LowerSetp[1]':(280.,310.),
                                'UpperSetp[1]':(280.,310.)}, 
        predictive_period    = 24*3600, 
        regressive_period    = 6*3600, 
        random_start_time    = True,
        max_episode_length   = 24*3600,
        warmup_period        = 24*3600,
        step_period          = 3600)

# Normalize observations and discretize action space
env = NormalizedObservationWrapper(env)
env = DiscretizedActionWrapper(env,n_bins_act=10)

# Instantiate an RL agent
model = DQN('MlpPolicy', env, verbose=1, gamma=0.99,
            learning_rate=5e-4, batch_size=24, 
            buffer_size=365*24, learning_starts=24, train_freq=1)

# Main training loop
model.learn(total_timesteps=10)

# Loop for one episode of experience (one day)
done = False
obs, _ = env.reset()
while not done:
  action, _ = model.predict(obs, deterministic=True) 
  obs,reward,terminated,truncated,info = env.step(action)
  done = (terminated or truncated)

# Obtain KPIs for evaluation
env.get_kpis()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)